# Install Dependencies

# Install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions

In [1]:
# !pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# import opencv as cv
import torch
# assert torch.__version__.startswith("1.7")
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtimeK

1.0.1 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



# Setup detectron2 logger

In [2]:
import detectron2
# from detectron2.utils.logger import setup_logger
# setup_logger()

# Import some common libraries


In [3]:
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

ModuleNotFoundError: No module named 'cv2'